在 LangChain 中，可以通过使用 `Memory` 模块为 Agent 增加记忆功能。记忆允许 Agent 在对话中保留上下文，从而实现更自然的交互。以下是实现步骤：

### 1. 引入必要的模块
需要引入 `ConversationBufferMemory` 或其他类型的记忆模块，以及 Agent 和工具。

```python
from langchain.memory import ConversationBufferMemory
from langchain.agents import initialize_agent, Tool
from langchain.chat_models import ChatOpenAI
```

### 2. 定义记忆
创建一个记忆对象，例如 `ConversationBufferMemory`，它会存储对话的上下文。

```python
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
```

### 3. 初始化 Agent
将记忆对象传递给 Agent，使其能够在对话中使用记忆。

```python
llm = ChatOpenAI(temperature=0)
tools = []  # 定义你的工具列表

agent = initialize_agent(
  tools=tools,
  llm=llm,
  agent="chat-conversational-react-description",
  memory=memory,
  verbose=True
)
```

### 4. 使用 Agent
与 Agent 交互时，它会自动利用记忆存储和检索对话上下文。

```python
response = agent.run("你好！")
print(response)

response = agent.run("还记得我刚才说了什么吗？")
print(response)
```

### 其他类型的记忆
LangChain 提供了多种记忆类型，可以根据需求选择：
- `ConversationBufferMemory`: 简单的对话缓冲区。
- `ConversationSummaryMemory`: 对话摘要记忆。
- `ConversationKnowledgeGraphMemory`: 知识图谱记忆。

### 示例代码
以下是完整的代码示例：

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.agents import initialize_agent, Tool
from langchain.chat_models import ChatOpenAI

# 初始化记忆
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# 初始化 LLM 和工具
llm = ChatOpenAI(temperature=0)
tools = []  # 工具列表

# 初始化 Agent
agent = initialize_agent(
  tools=tools,
  llm=llm,
  agent="chat-conversational-react-description",
  memory=memory,
  verbose=True
)

# 与 Agent 交互
response = agent.run("你好！")
print(response)

response = agent.run("还记得我刚才说了什么吗？")
print(response)

#### 1 把Memory添加到提示词模板中

In [2]:
from bocha_api import bocha_websearch_tool as websearch_tool

In [58]:
from langchain.prompts import PromptTemplate
from langchain.prompts import MessagesPlaceholder
from langchain.agents import load_tools, AgentType, initialize_agent
from langchain_ollama.chat_models import ChatOllama
from langchain.memory import ConversationBufferMemory
from langchain.agents import Tool

# 初始化 LLM 和工具
llm = ChatOllama(model="deepscaler", temperature=0)

# 定义工具
# 创建LangChain工具
bocha_web_tool = Tool(
    name="BochaWebSearch",
    func=websearch_tool,
    description="使用Bocha Web Search API 进行搜索互联网网页，输入应为搜索查询字符串，输出将返回搜索结果的详细信息，包括网页标题、网页URL、网页摘要、网站名称、网站Icon、网页发布时间等。"
)

# 定义工具列表
tools = load_tools(
    tool_names=["llm-math"],
    llm=llm
)

# 将自定义工具添加到工具列表中
tools.append(bocha_web_tool)

*添加Memory记忆*

In [59]:
# 初始化记忆
# 这里需要注意一点 默认的 memory_key 的占位是在模板固定写死的，可能需要根据Agent修改
# memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
# 修改为MemoryKey
memory = ConversationBufferMemory(memory_key="memory", return_messages=True)

*定义Agent*

In [61]:
agent_chain = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    memory=memory,
    verbose=True
)

In [62]:
# 看一下默认的Agent的Prompt
print(agent_chain.agent.prompt.messages[0])
print(agent_chain.agent.prompt.messages[1])
print(agent_chain.agent.prompt.messages[2])

content='You are a helpful AI assistant.' additional_kwargs={} response_metadata={}
prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}') additional_kwargs={}
variable_name='agent_scratchpad'


In [95]:
# 修改Agent
agent_chain = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    # 使用agent_kwargs来修改Agent的Prompt
    # 这里的agent_kwargs是一个字典
    agent_kwargs={
        # 这里把 variable_name='agent_scratchpad'修改为variable_name='memory'
        # 然后还需要再把 agent_scratchpad的值传入到Prompt中
        # "extra_prompt_messages" 这个名称是固定的
        "extra_prompt_messages": [
            MessagesPlaceholder(variable_name="memory"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ],
    },
    memory=memory,
)

In [96]:
# 再看一下修改的Agent的Prompt
# print(agent_chain.agent.prompt.messages)
print(agent_chain.agent.prompt.messages[0])
print(agent_chain.agent.prompt.messages[1])
print(agent_chain.agent.prompt.messages[2])
# 前面定义的MemoryKey中为memory， 对话记录会拆入到该位置

content='You are a helpful AI assistant.' additional_kwargs={} response_metadata={}
variable_name='memory'
variable_name='agent_scratchpad'
